## Лабораторная работа NN-1: Разработка программы, которая обучает искусственную нейронную сеть (персептрон) распознавать два или более черно-белых изображения.

### Выполнил: Журавлев Д. А. Группа 211-321 

Учебный курс: Методы работы с большими данными

<details><summary>Описание ЛР</summary>

### 1. Цель работы
Изучить принципы работы и алгоритм обучения простейших искусственных нейронных сетей (НС).

### 2. Порядок выполнения работы
1. Изучить теоретическое введение.
2. Сформировать обучающую выборку из 10+ изображений.
3. Разработать компьютерную программу (среда разработки выбирается студентом самостоятельно).
4. Провести серию из 5+ испытаний с различными исходными данными, выявить ограничения и недостатки однослойных НС для решения задач распознавания.
5. Оформить отчет по лабораторной работе.

### 3. Требования к исходным данным и функциональности компьютерной программы
- В программе должна быть реализована возможность задания обучающей выборки из внешних файлов изображений.
- Изображения должны быть черно-белыми (bitmap) и размером не менее 9 (3x3) пикселей.
- Программа должна иметь два режима работы: обучения и распознавания.
- Обучение должно производиться по стандартному алгоритму обучения персептрона с использованием дельта-правила.
- В программе должны задаваться следующие настройки:
  - количество входов нейрона, которое соответствует общему числу пикселей изображения
  - коэффициент скорости обучения (если его значение постоянно)
  - правильные варианты элементов обучающей выборки
  - размер ошибки, при котором обучение персептрона завершается (опционально)

#### На экранной форме режима обучения должны отображаться:
- элементы обучающей выборки (изображения)
- настройки алгоритма обучения
- текущие (итоговые) веса нейронов и значение порога активационной функции
- протоколы результатов обучения (значения весов для каждой итерации)

#### На экранной форме режима распознавания должны отображаться:
- распознаваемое изображение (должно выбираться из всего множества)
- результат распознавания
- веса нейронов и значение порога активационной функции
- значения выходов всех нейронов до и после применения активационной функции

### 4. Рекомендации по реализации
- Для задания различной размерности распознаваемых изображений можно пользоваться одним типо-размером с максимальной разрешающей способностью, но при этом считывать только часть пикселей (например, от верхнего левого угла).
- Для решения задач обучения двухмерное изображение N*M можно преобразовывать в одномерный вектор (массив) размерностью K=N*M.
- При распознавании цветных изображений (RGB) каждому пикселю соответствует 3-х байтовая последовательность (24 входа).

### 5. Содержание отчета
- Название и цель работы
- Задание, краткое описание предметной области и выбранной задачи
- Блок-схема алгоритмов обучения и распознавания
- Протоколы проведенных экспериментов (5+), представленные в форме таблиц и графиков (допускаются скриншоты в случае программной реализации этой функциональности)
- Выводы и рекомендации по использованию НС для решения задач распознавания
</details>

## Краткое описание предметной области и выбранной задачи

**Предметная область**:  
Искусственные нейронные сети (ИНС) представляют собой вычислительные модели, вдохновленные структурой и функционированием человеческого мозга. Персептрон, как одна из самых простых моделей нейронной сети, используется для решения задач классификации. ИНС применяются в различных областях, включая компьютерное зрение, обработку изображений, распознавание объектов и много других.

**Задача**:  
Целью данной работы является разработка программы, обучающей искусственную нейронную сеть (персептрон) для распознавания черно-белых изображений, содержащих два или более простых геометрических объекта, таких как круги, квадраты, треугольники. Программа должна уметь распознавать эти объекты на изображениях размером 5x5 пикселей.

Задача будет решаться путем:
1. Сборки тренировочных данных, состоящих из набора изображений.
2. Обучения персептрона на этих изображениях.
3. Оценки качества работы сети на тестовых данных и контрольных данных.

В процессе работы необходимо будет реализовать:
- Подготовку и нормализацию данных (изображений).
- Архитектуру нейронной сети (персептрон) с одним скрытым слоем.
- Функцию активации для нейронов.
- Процесс обучения с использованием алгоритма обратного распространения ошибки (backpropagation).

## Блок-схема работы алгоритма

![Блок-схема алгоритма](./diagrams/NN-1-diagram.png)

### Описание блок-схемы алгоритмов обучения и распознавания перцептрона

1. **Инициализация перцептрона**:  
   Устанавливаются ключевые параметры: размер входных данных, число классов, скорость обучения. Генерируются случайные значения для весов и смещений.

2. **Цикл обучения**:  
   - **Вычисление выходных значений**:  
     Используется сигмоидная функция активации для получения вероятностных предсказаний.  
   - **Вычисление ошибки**:  
     Определяется разница между ожидаемым результатом и текущими предсказаниями.  
   - **Корректировка весов и смещений**:  
     Градиентный спуск обновляет параметры модели на основе производной функции активации и текущей ошибки.  
   - **Проверка условий завершения**:  
     Алгоритм прерывается, если ошибка становится меньше заданного минимума или достигнуто предельное количество эпох.  

3. **Сохранение модели**:  
   Сохраняются обученные значения весов и смещений для дальнейшего использования.

4. **Распознавание**:  
   - **Загрузка модели**:  
     Используются ранее сохраненные веса и смещения.  
   - **Предсказание класса**:  
     Входные данные преобразуются через линейную комбинацию весов, добавление смещений и функцию активации.  
   - **Выбор класса**:  
     Результирующий класс определяется как индекс с максимальной вероятностью в выходном векторе.  

Блок-схема подчеркивает взаимодействие между обучением и распознаванием: модель сначала оптимизирует параметры на основе данных, а затем применяет их для классификации новых объектов.

## Настройка параметров алгоритма, импорт и инициализация зависимостей
 
- **TRAINING_IMAGES_DIR**: путь к директории, где хранятся изображения для обучения.  
- **WEIGHTS_DIR**: путь к директории, где будут сохраняться веса модели после обучения.  
- **IMAGE_SIZE**: размер изображений, которые будут использоваться для обучения (50x50 пикселей).  
- **LEARNING_RATE**: скорость обучения нейронной сети, контролирующая шаг изменения весов.  
- **MIN_ERROR**: минимально допустимая ошибка для завершения процесса обучения.  

In [ ]:
from PIL import Image
import numpy as np
import os
import csv

np.random.seed(6341)

TRAINING_IMAGES_DIR = "./materials/nn1/images/"
WEIGHTS_DIR = "./materials/nn1/weights/"
IMAGE_SIZE = (50, 50)
LEARNING_RATE = 0.1
MIN_ERROR = 0.035


## Загрузка изображений и разделение на группы

**Загрузка изображений**:
   - Используется метод **Image.open()** для открытия изображения и его преобразования в черно-белое с помощью **convert("L")**.
   - Изображения изменяются до заданного размера с помощью **resize(image_size)**.
   - Изображения преобразуются в бинарный формат с использованием **np.array(image) < 128**, где пиксели, яркость которых ниже 128, становятся единичными.
   - Затем используется метод **.flatten()**, чтобы превратить двухмерный массив в одномерный вектор.
   - С помощью **str(filename).rsplit(".", 1)[0].split("_")** извлекаются группы и имена файлов.

In [28]:
def load_and_vectorize_images(directory, image_size):
    train_vectors = []
    control_vectors = []
    for filename in os.listdir(directory):
        image = Image.open(os.path.join(directory, filename)).convert("L").resize(image_size)
        vector = (np.array(image) < 128).astype(int).flatten()
        group, name = str(filename).rsplit(".", 1)[0].split("_")
        data_entry = (group, f"{group}_{name}", vector)
        if name == "control":
            control_vectors.append(data_entry)
        else: 
            train_vectors.append(data_entry)
    return train_vectors, control_vectors

train_data, control_data = load_and_vectorize_images(TRAINING_IMAGES_DIR, IMAGE_SIZE)

print(f"Train data {train_data}\nControl data {control_data}")

Train data [('cross', 'cross_23', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_15', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_14', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_22', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_1', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_3', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_21', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_2', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_25', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_13', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_12', array([0, 0, 0, ..., 0, 0, 0])), ('cross', 'cross_24', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_5', array([0, 0, 0, ..., 0, 0, 0])), ('oval', 'oval_10', array([0, 0, 0, ..., 0, 0, 0])), ('polygon', 'polygon_11', array([0, 0, 0, ..., 0, 0, 0])), ('rectangle', 'rectangle_4', array([0, 0, 0, ..., 0, 0, 0])), ('oval', 'oval_8', array([0, 0, 0, ..., 0, 0, 0])), ('s

## Создание словаря меток групп

**group_labels** - словарь, который отображает уникальные значения группы в числовые метки.

In [29]:
group_labels = { y : x for x, y in enumerate( set( group[0] for group in train_data ) ) }

group_labels

{'polygon': 0, 'cross': 1, 'rectangle': 2, 'star': 3, 'oval': 4}

## Подготовка и разметка обучающех и контрольных дата-сетов

**create_labeled_data** - форматирует данные в формат, пригодный для обучения.

`group_labels`: словарь, который связывает метки групп с их числовыми идентификаторами.
`data`: кортеж формата `(group, name, vector)`

Результат работы функции:
- `vectors` - массив с векторами признаков
- `labels` - массив меток

In [30]:
def create_labeled_data(group_labels: dict, data: list):
    vectors = []
    labels = []
    
    for group, _, vector in data:
        vectors.append(np.array(vector))
        labels.append(group_labels[group])
    
    return np.array(vectors), np.eye(len(group_labels), dtype=int)[labels]

train_vectors, train_labels = create_labeled_data(group_labels, train_data)
control_vectors, control_labels = create_labeled_data(group_labels, control_data)

print(f"Train vectors\n {train_vectors}\nTrain labels\n {train_labels}\n Control vectors\n {control_vectors}\n Contrpl labels\n {control_labels}")

Train vectors
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Train labels
 [[0 1 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
 Control vectors
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
 Contrpl labels
 [[1 0 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]]


## Определение класса Перцептрон

**Perceptron** — класс, реализующий нейронную сеть с одним слоем для решения задачи классификации.

- В конструкторе инициализируются параметры:
  - `input_size`: размерность входного вектора.
  - `output_size`: количество классов.
  - `learning_rate`: коэффициент обучения.
  - `weights`: случайно инициализированные веса.
  - `bias`: случайно инициализированные смещения.

### Методы:
- **activation**: сигмоидная функция активации.
- **activation_derivative**: производная от сигмоидной функции для вычисления градиента.
- **train**: процесс обучения:
  - Рассчитывает выход сети.
  - Вычисляет ошибку и её производную.
  - Обновляет веса и смещения с учетом ошибки и производной.
  - Прекращает обучение, если ошибка меньше заранее заданного порога.
  - Записывает веса и смещения в CSV файлы.
- **predict**: делает предсказание, возвращая индекс класса с наибольшим значением активации.

### Процесс обучения:
- Персептрон инициализируется случайными значениями весов и смещений.
- На каждой эпохе данные проходят через сеть, и вычисляется ошибка между предсказанием и истинными метками.
- Веса и смещения обновляются с помощью градиентного спуска, чтобы минимизировать ошибку.
- Обучение продолжается до достижения минимальной ошибки или заданного числа эпох.
- После обучения модель сохраняет веса и смещения, которые могут быть использованы для предсказаний.

In [31]:
class Perceptron:
    def __init__(self, input_size, output_size, learning_rate):
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.weights = np.random.randn(input_size, output_size)
        self.bias = np.random.randn(output_size)

    def activation(self, x):
        return 1 / (1 + np.exp(-x))

    def activation_derivative(self, x):
        return x * (1 - x)

    def train(self, X, y, epochs=10000):
        for epoch in range(epochs):
            output = self.activation(np.dot(X, self.weights) + self.bias)
            error = y - output
            d_output = error * self.activation_derivative(output)
            self.weights += np.dot(X.T, d_output) * self.learning_rate
            self.bias += np.sum(d_output, axis=0) * self.learning_rate
            loss = np.mean(np.abs(error))

            if epoch % 1000 == 0: 
                print(f"Epoch {epoch}, Loss: {loss}")

            if np.mean(np.abs(error)) <= MIN_ERROR:
                print(f"Epoch {epoch}, Loss: {loss}: Mininal error achived")
                break
        
        with open(WEIGHTS_DIR + 'weights.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(self.weights)

        with open(WEIGHTS_DIR + 'bias.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows([self.bias])

    def predict(self, X):
        output = self.activation(np.dot(X, self.weights) + self.bias)
        return np.argmax(output)
    
perceptron = Perceptron(
    IMAGE_SIZE[0] * IMAGE_SIZE[1],
    len(group_labels.keys()),
    LEARNING_RATE
)

## Обучение персептрона на тренировочных данных

`train` инициирует процесс обучения персептрона.  

- **Данные**: `train_vectors` (входные признаки) и `train_labels` (истинные метки).  
- **Количество эпох**: До 100,000 или до достижения минимальной ошибки (`MIN_ERROR`).  

In [32]:
perceptron.train(train_vectors, train_labels, epochs=100000)

Epoch 0, Loss: 0.5566738082059293
Epoch 1000, Loss: 0.07718577301332545
Epoch 2000, Loss: 0.05987837636495278
Epoch 3000, Loss: 0.0590956546376545
Epoch 4000, Loss: 0.05868086928429081
Epoch 5000, Loss: 0.05840553723164399
Epoch 6000, Loss: 0.0582039937620272
Epoch 7000, Loss: 0.058047493010700044
Epoch 8000, Loss: 0.05792095524832245
Epoch 9000, Loss: 0.05781552512984927
Epoch 10000, Loss: 0.05772555335466913
Epoch 11000, Loss: 0.05764715602530095
Epoch 12000, Loss: 0.05757737128652516
Epoch 13000, Loss: 0.05751320100237702
Epoch 14000, Loss: 0.05739529695924873
Epoch 15000, Loss: 0.04944409466445958
Epoch 16000, Loss: 0.04938339693481435
Epoch 17000, Loss: 0.04933530724250619
Epoch 18000, Loss: 0.04928946204698151
Epoch 19000, Loss: 0.041334048086100604
Epoch 20000, Loss: 0.04126772092551489
Epoch 21000, Loss: 0.041215965609566314
Epoch 21523, Loss: 0.03494540009814562: Mininal error achived


## Вычисление точности предсказаний

**`calc_accuracy`**: функция для оценки точности работы персептрона.  

1. **Аргументы**:
   - `vectors`: входные данные, используемые для предсказаний.
   - `labels`: истинные метки в формате one-hot.

2. **Процесс**:
   - Для каждой пары векторов и меток:
     - Использует метод `predict` для получения предсказанной группы.
     - Преобразует предсказание и метку из числовых индексов обратно в групповые названия.
     - Сравнивает предсказанную и ожидаемую группы, фиксируя корректность.

In [33]:

def calc_accuracy(vectors, labels):
    correct_predictions = 0
    for vector, label in zip(vectors, labels):
        prediction = perceptron.predict(vector)
        predicted_group = [ i for i in group_labels if group_labels[i]==prediction ][0]
        expected_group = [ i for i in group_labels if group_labels[i]==np.where(label == 1)[0][0] ][0]
        prediction_correction = "✅" if expected_group == predicted_group else "❌"
        print(f"{prediction_correction} Prediction: {predicted_group}, expected: {expected_group}")
        if expected_group == predicted_group:
            correct_predictions += 1

    return correct_predictions / len(vectors) * 100

print(f"Точность на обучающих данных: {calc_accuracy(train_vectors, train_labels):.2f}%\n")
print(f"Точность на контрольных данных: {calc_accuracy(control_vectors, control_labels):.2f}%\n")

✅ Prediction: cross, expected: cross
✅ Prediction: polygon, expected: polygon
✅ Prediction: polygon, expected: polygon
✅ Prediction: cross, expected: cross
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: cross, expected: cross
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: cross, expected: cross
✅ Prediction: polygon, expected: polygon
✅ Prediction: polygon, expected: polygon
✅ Prediction: cross, expected: cross
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: oval, expected: oval
✅ Prediction: polygon, expected: polygon
✅ Prediction: rectangle, expected: rectangle
✅ Prediction: oval, expected: oval
✅ Prediction: star, expected: star
✅ Prediction: star, expected: star
✅ Prediction: oval, expected: oval
✅ Prediction: star, expected: star
❌ Prediction: star, expected: oval
❌ Prediction: polygon, expected: star
✅ Prediction: star, expected: star
❌ Prediction: rectangle, expected: oval
Точность на обучающих данны

## Выводы по использованию перцептрона для задач распознавания

1. **Эффективность для простых задач**:  
   Перцептрон подходит для задач распознавания с линейно разделимыми классами, таких как базовая классификация изображений или бинарные задачи.

2. **Ограничения архитектуры**:  
   Однослойный перцептрон не справляется с нелинейно разделимыми данными (например, задача XOR). Для таких случаев требуется многослойная архитектура.

3. **Потребность в качественной подготовке данных**:  
   Перцептрон чувствителен к масштабированию и нормализации входных данных. Необходима предварительная обработка, включая преобразование изображений в бинарные векторы.

4. **Скорость сходимости**:  
   - Скорость обучения зависит от выбора параметров, таких как скорость обучения и допустимая ошибка.  
   - При слишком малых значениях скорость обучения может замедляться, а при больших – привести к нестабильности.

5. **Хранение результатов**:  
   Сохранение обученных весов и смещений позволяет повторно использовать модель без необходимости повторного обучения, что особенно важно для ресурсоемких задач.

6. **Обобщающая способность**:  
   Модель демонстрирует хорошие результаты на данных, аналогичных обучающим. Однако на контрольных данных из отличных распределений точность может снижаться.

7. **Практическая применимость**:  
   Использование перцептрона оправдано для небольших задач с ограниченным числом классов. Для более сложных задач рекомендуется переходить к глубоким нейронным сетям.